In [21]:
import json
import requests
import re
import nltk
import os 
from dotenv import load_dotenv
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from rake_nltk import Rake
#nltk.download('averaged_perceptron_tagger')
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('vader_lexicon')

In [22]:
def searchTweets(search_term:str):
    load_dotenv()
    url = "https://api.twitter.com/2/tweets/search/recent?max_results=25&expansions=referenced_tweets.id&tweet.fields=text&query=lang%3Aen%20"+search_term

    payload={}
    headers = {
    'Authorization': os.getenv('BEARER_TOKEN'),
    'Cookie': 'guest_id=v1%3A165292464833657250; guest_id_ads=v1%3A165292464833657250; guest_id_marketing=v1%3A165292464833657250; personalization_id="v1_D50leSEsdlQN9nTvwQ6B+g=="'
    }
    response = requests.request("GET", url, headers=headers, data=payload).json()['includes']['tweets']

    tweets = [i['text'] for i in response]


    out_file = open("myfile.json", "w") 
    json.dump(tweets, out_file, indent = 6) 
    out_file.close() 
    
    with open('myfile.json') as f:
     data = json.load(f)

    return data

In [23]:
tweets = searchTweets("Biden")

In [24]:
print(tweets)

['Pennsylvania Lt. Gov. John Fetterman is calling on President Biden to deschedule marijuana before his visit to Pennsylvania next week.', 'The laptop from hell was buried so deeply because it’s not about Hunter Biden—it’s about the corruption of Joe Biden.', 'Joe Biden raped me RT', 'Biden is raising your taxes\nBiden wants your guns\nBiden doesn’t care about securing the border.\nBiden has made us dependent on bad actors for energy.\nBiden has created record-breaking inflation.\nBiden has yet to ‘create’ a single job.\n\nVote accordingly come November.', "If Biden thinks that the Patriots need F-15's to keep our Freedom then someone needs to get that man checked into a Mental Hospital. This is America the Land of the Free! 🙄\U0001fae3", 'OMG they let creepy Joe Biden out of the basement today and he was a complete bumbling stumbling angry weirdo 🤣🤣🤣🤣🤣', 'If Trump said something like this the media would absolutely crucify him. Biden? Not so much. \n\nhttps://t.co/9cO9PYXroJ', 'Walker

In [25]:
#Build text pre-processing 
#would a class for the tweets be smart?

### Text Preprocessing

In [26]:

def process_text(text):
    stop_words = set(stopwords.words('english'))
    clean_text = re.sub(r'\b(https?://\S+)\b','',text) #remove URLs
    clean_text = " ".join(re.findall('\w{3,}',clean_text))
    word_tokens = word_tokenize(clean_text)
    filtered_text = " ".join([word for word in word_tokens if word not in stop_words])
    
    return filtered_text

In [27]:
clean_tweets = [process_text(i) for i in tweets]

In [28]:
#get keywords
clean_tweets

['Pennsylvania Gov John Fetterman calling President Biden deschedule marijuana visit Pennsylvania next week',
 'The laptop hell buried deeply Hunter Biden corruption Joe Biden',
 'Joe Biden raped',
 'Biden raising taxes Biden wants guns Biden care securing border Biden made dependent bad actors energy Biden created record breaking inflation Biden yet create single job Vote accordingly come November',
 'Biden thinks Patriots need keep Freedom someone needs get man checked Mental Hospital This America Land Free',
 'OMG let creepy Joe Biden basement today complete bumbling stumbling angry weirdo',
 'Trump said something like media would absolutely crucify Biden Not much',
 'Walker bike bent anyone ride bike like seem Chuck Schumer Joe Biden riding bike',
 'Trump wants Hunter Biden charged crime Trump would Trump would lost 2016 Comey raised false flag Hillary Losers keep making excuses wtpBLUE ProudBlue22',
 'Between COVID taking endless vacation Biden largely sight weeks leading many for

In [29]:
from statistics import mean
from math import floor

def get_keywords(sentences: list):
    '''
    Given a list of strings, return the top keywords that are at least 4 letters long
    '''        
    r = Rake()
    r.extract_keywords_from_sentences(sentences)
    r.get_ranked_phrases_with_scores()
    keywords = r.get_ranked_phrases()[0].split(" ")
    avg_keyword_len = floor(mean([len(i) for i in keywords]))
    best_keywords = [i for i in keywords if len(i)>avg_keyword_len and len(i)>3]
    return best_keywords

In [30]:
get_keywords(clean_tweets)


['raising',
 'securing',
 'border',
 'dependent',
 'actors',
 'energy',
 'created',
 'record',
 'breaking',
 'inflation',
 'create',
 'single',
 'accordingly',
 'november']

In [31]:
#get sentiment
from nltk.sentiment import SentimentIntensityAnalyzer
def get_avg_sentiment(tweets):
    sia = SentimentIntensityAnalyzer()

    term_sentiment = round(mean([sia.polarity_scores(tweet)['compound'] for tweet in tweets ]),2)

    if term_sentiment < -0.33:
        return ("negative", term_sentiment)
    elif term_sentiment > 0.33:
        return ("positive", term_sentiment)
    else:
        return ("neutral",term_sentiment)

get_avg_sentiment(clean_tweets)


('neutral', -0.05)

In [38]:
keywords = get_keywords(clean_tweets,)

tag=pos_tag(keywords)

[i for i in tag if i[1] not in ['VBG','VBD','RB']]

[('border', 'NN'),
 ('dependent', 'JJ'),
 ('actors', 'NNS'),
 ('energy', 'NN'),
 ('record', 'NN'),
 ('inflation', 'NN'),
 ('create', 'NN'),
 ('single', 'JJ'),
 ('november', 'VBP')]